### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

### Testing

In [3]:
list(ec2.key_pairs.all())

[ec2.KeyPairInfo(name='AmazonEC2test'),
 ec2.KeyPairInfo(name='aws-key-fast-ai'),
 ec2.KeyPairInfo(name='aws-key-test-vpc'),
 ec2.KeyPairInfo(name='id_rsa'),
 ec2.KeyPairInfo(name='Mac')]

#### Define parameters

In [4]:
vpc_name='fast-ai'

#### Get Existing VPC by tag name

In [5]:
vpc = get_vpc(vpc_name); vpc

ec2.Vpc(id='vpc-6e6b2a17')

#### Request Spot instance

In [6]:
instance_name = f'{vpc_name}-instance'
instance_type = 't2.micro'

In [7]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.003500'

In [8]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [9]:
launch_specs['BlockDeviceMappings'][0]['Ebs']['VolumeSize'] = 80

In [10]:
launch_specs

{'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'VolumeSize': 80,
    'VolumeType': 'gp2'}}],
 'ImageId': 'ami-8c4288f4',
 'InstanceType': 't2.micro',
 'KeyName': 'aws-key-fast-ai',
 'NetworkInterfaces': [{'AssociatePublicIpAddress': True,
   'DeviceIndex': 0,
   'Groups': ['sg-f60fca88'],
   'SubnetId': 'subnet-f056ff89'}]}

In [11]:
instance = create_spot_instance(instance_name, launch_specs, spot_price='0.5'); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0d10b8fa3b383efb3
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@54.202.16.120


ec2.Instance(id='i-0d10b8fa3b383efb3')

Request on demand instance (if spot error)

In [ ]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [ ]:
instance = get_instance(instance_name); instance

#### Attach EBS volume (Optional)

In [12]:
volume_tag = 'fast-ai-ebs-volume-test'

In [13]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

Could not find volume for tag: fast-ai-ebs-volume-test


### SSH

In [45]:
client = connect_to_instance(instance)

Connecting to SSH...
Exception: timed out Retrying...
Exception: timed out Retrying...
Exception: timed out Retrying...
Exception: [Errno None] Unable to connect to port 22 on 34.217.61.190 Retrying...
Connected!


#### Mount EBS

In [ ]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

In [ ]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

In [ ]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

In [ ]:
out, _ = run_command(client, 'ls ebs_mount_point') # no reformatting

#### Mount EFS

In [48]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [46]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [49]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [50]:
out, _ = run_command(client, efs_mount_cmd)

run_command returned: 



In [51]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt

